<a href="https://colab.research.google.com/github/Ayumi-Smith/ADS/blob/main/Kopia_notatnika_M8PD1V1_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modul 8 - Praca domowa - Python

Zwykle istnieje wiele sposobów, żeby rozwiązać zadanie w Python. Także czasami warto spróbować opracować kod na 1-2 sposoby :)

## Zadanie - 1

<img src="https://media.giphy.com/media/AgHBbekqDik0g/giphy.gif" width="800">

> Wykorzystaj platwormę Bankier.pl do pobrania tabeli z notowaniami [[Źródło Danych](https://bit.ly/3boWGJ7)]. Dane dotyczą notowań cen akcji na GPW. Zescrapuj całą tabelę z notowaniami wykorzystując:





>**1. Podejście - biblioteki: `requests` oraz `BeautifulSoup`**


In [2]:
import requests
from bs4 import BeautifulSoup

r = requests.get('https://www.bankier.pl/gielda/notowania/akcje')
soup = BeautifulSoup(r.text, 'html.parser')
table = soup.find_all('table')[0]
#print(table.text)

>**2. Podejście - biblioteka: `pandas`**


In [3]:
import pandas as pd
df = pd.read_html('https://www.bankier.pl/gielda/notowania/akcje')[0]
print(df.head())

   Walor AD  Kurs AD  Zmiana AD Zmiana procentowa AD Liczba transakcji AD  \
0   06MAGNA    28700      300.0                1,06%                   38   
1  08OCTAVA    08900        0.0                0,00%                    2   
2     11BIT  2510000   -45000.0               -1,76%                  853   
3   3RGAMES    02970      -30.0               -1,00%                   11   
4      ABPL   890000        0.0                0,00%                   20   

    Obrót AD Otwarcie AD   Max AD   Min AD      Czas AD  
0     27 980       28400    28700    27500  29.11 17:00  
1         20       08900    08900    08900  29.11 15:00  
2  3 894 726     2590000  2590000  2475000  29.11 17:04  
3      7 242       03000    03000    02890  29.11 17:00  
4     19 652      890000   898000   890000  29.11 16:31  


>**Zdefiniuj odpowiedzi na pytania:**
* Która z akcji obecnie jest najdroższa na GPW?
* Top 5 akcji, które odnotowały największy wzrost wartości akcji w ostatnim okresie?
* Top 5 akcji, które odnotowały największy spadek wartości akcji w ostatnim okresie?
* Inne przydatne informacje (*opcjonalnie*)

In [36]:
df['Kurs AD']=df['Kurs AD'].str.replace(',','.')
df['Kurs AD'] = df['Kurs AD'].str.replace('\xa0', '')
df['Kurs AD'] = df['Kurs AD'].fillna(0)
df['Kurs AD']=df['Kurs AD'].astype(float)


In [38]:
najdrozsza = df['Kurs AD'].sort_values(ascending=False).head(1)
print(najdrozsza)

371    8.62
Name: Zmiana procentowa AD, dtype: object


In [41]:
wzrost = df['Zmiana procentowa AD'].str.replace('%','').str.replace(',','.').astype(float).sort_values(ascending=False).head(5)
print(wzrost)



286    46.97
114    22.81
98     20.81
179    11.02
41     10.47
Name: Zmiana procentowa AD, dtype: float64


In [42]:
spadek = df['Zmiana procentowa AD'].str.replace('%','').str.replace(',','.').astype(float).sort_values(ascending=True).head(5)
print(spadek)

167   -93.64
305   -32.50
171   -15.62
350   -15.15
31    -12.41
Name: Zmiana procentowa AD, dtype: float64


## Zadanie - 2

<img src="https://media.giphy.com/media/AbDCwAI2xTwTm/giphy.gif" width="500">

> Wykorzystaj platwormę Filmweb.pl do pobrania informacji o najlepszych 25 filmach z rankingu 500 filmów z całego świata [[Źródło Danych](https://bit.ly/3bnWsSF)]. Strona zawiera informacje: pozycja w rankinhu, nazwa filmu po polsku, nazwa filmu po angielsku, rok premiery, średnia ocena, liczba ocen, itd. Zrealizuj kolejne zadania:

>1. Zescrapuj 25 pierwszych filmów z rankingu, pobierając informacje o: pozycja w rankinhu, nazwa filmu po polsku, nazwa filmu po angielsku, rok premiery, średnia ocena, liczba ocen, link do miniaturki, link do podstrony.

In [4]:
r = requests.get('https://bit.ly/3bnWsSF')
soup = BeautifulSoup(r.text, 'html.parser')

films_div = soup.find('div', class_='page__container rankingTypeSection__container')
films = films_div.find_all('div', itemprop="itemListElement")
names = []
urls = []
positions = []
ratings = []
votes = []
en_names = []
images = []

for film in films:
    name = film.find('a', itemprop="url").text
    names.append(name)
    url = film.find('a', itemprop="url")['href']
    urls.append('https://www.filmweb.pl'+url)
    position = film.find('span', itemprop="position").text.replace('.','')
    positions.append(position)
    rating = film.find('span', itemprop="ratingValue").text
    ratings.append(rating)
    vote = film.find('span', itemprop="ratingCount").text.replace('ocen','').replace(' y','')
    votes.append(vote)
    en_name = film.find('p', class_="rankingType__originalTitle").text
    en_names.append(en_name)
    image = film.find('span', itemprop="image").text
    images.append(image)
    #print(name, url, position, rating, votes, year, en_name, image)

>2. Stwórz DF oraz uzupełnij pobrane linki, jeżeli nie są pobrane w całości.

In [12]:
df = pd.DataFrame()
df['Name'] = names
df['URL'] = urls
df['Position in rating'] = positions
df['Rating'] = ratings
df['Votes count'] = votes
df['en_name'] = en_names
df['Image'] = images
df.set_index('Position in rating', inplace=True)
df[['English name','Year']] = df['en_name'].str.rsplit(' ', n=1, expand=True)
df.drop('en_name', axis=1, inplace=True)
df.head()

,Name,URL,Rating,Votes count,Image,English name,Year
Position in rating,,,,,,,
1,Skazani na Shawshank,https://www.filmweb.pl/film/Skazani+na+Shawsha...,"8,77",937 331,https://fwcdn.pl/fpo/10/48/1048/6925401_1.7.webp,The Shawshank Redemption,1994
2,Zielona mila,https://www.filmweb.pl/film/Zielona+mila-1999-862,"8,60",1 012 360,https://fwcdn.pl/fpo/08/62/862/7517878_1.7.webp,The Green Mile,1999
3,Nietykalni,https://www.filmweb.pl/film/Nietykalni-2011-58...,"8,60",689 064,https://fwcdn.pl/fpo/33/90/583390/7441162_1.7....,Intouchables,2011
4,Ojciec chrzestny,https://www.filmweb.pl/film/Ojciec+chrzestny-1...,"8,59",555 526,https://fwcdn.pl/fpo/10/89/1089/7196615_1.7.webp,The Godfather,1972
5,Dwunastu gniewnych ludzi,https://www.filmweb.pl/film/Dwunastu+gniewnych...,"8,58",233 079,https://fwcdn.pl/fpo/07/01/30701/8094290_1.7.webp,12 Angry Men,1957


>3. Wykorzystując linki do podstron z listy pobranych filmów, uzupełnij stworzony DF o informacje: opis filmu, informacje o nagrodach, czas trwania filmu, reżyser, scenariusz, gatunek, produkcja.

In [38]:
for row in df.iterrows():
    r = requests.get(row[1]['URL'])
    soup = BeautifulSoup(r.text, 'html.parser')
    awards_container = soup.find('div', class_='filmPosterSection__buttons tags-container')
    award = awards_container.find_all('div', class_='LinkButton linkButton')[1]
    print(award.text)



  Top 1994 roku #1 
  Top 1999 roku #1 
  Top 2011 roku #1 
  Top 1972 roku #1 
  Top 1957 roku #1 
  Top 1994 roku #2 
  Top 1975 roku #1 


KeyboardInterrupt: 

In [39]:
descriptions = []
awards = []
durations = []
directors = []
writers = []
genres = []
productions = []

for row in df.iterrows():
    r = requests.get(row[1]['URL'])
    soup = BeautifulSoup(r.text, 'html.parser')
    description = soup.find('span', class_='description').text
    descriptions.append(description)
    awards_container = soup.find('div', class_='filmPosterSection__buttons tags-container')
    award = awards_container.find_all('div', class_='LinkButton linkButton')[1]
    awards.append(award.text)
    duration = soup.find('div', itemprop='timeRequired')
    durations.append(duration.text)
    director = soup.find('a', itemprop='director')
    directors.append(director.text)
    writer = soup.find('a', itemprop='creator')
    writers.append(writer.text)
    genre = soup.find('div', class_='LinkButton linkButton linkButton--white')
    genres.append(genre.text)
    production = soup.find('div', class_='filmInfo__info filmInfo__info--productionCountry')
    productions.append(production.text)

df['Description'] = descriptions
df['Awards'] = awards
df['Duration'] = durations
df['Director'] = directors
df['Writer'] = writers
df['Genre'] = genres
df['Production'] = productions

df.head()




,Name,URL,Rating,Votes count,Image,English name,Year,Description,Duration,Director,Writer,Genre,Production,Awards
Position in rating,,,,,,,,,,,,,,
1,Skazani na Shawshank,https://www.filmweb.pl/film/Skazani+na+Shawsha...,"8,77",937 331,https://fwcdn.pl/fpo/10/48/1048/6925401_1.7.webp,The Shawshank Redemption,1994,Adaptacja opowiadania Stephena Kinga. Niesłusz...,2h 22m,Frank Darabont,Frank Darabont,Dramat,USA,Top 1994 roku #1
2,Zielona mila,https://www.filmweb.pl/film/Zielona+mila-1999-862,"8,60",1 012 360,https://fwcdn.pl/fpo/08/62/862/7517878_1.7.webp,The Green Mile,1999,Emerytowany strażnik więzienny opowiada przyja...,3h 8m,Frank Darabont,Frank Darabont,Dramat,USA,Top 1999 roku #1
3,Nietykalni,https://www.filmweb.pl/film/Nietykalni-2011-58...,"8,60",689 064,https://fwcdn.pl/fpo/33/90/583390/7441162_1.7....,Intouchables,2011,Sparaliżowany milioner zatrudnia do opieki mło...,1h 52m,Olivier Nakache,Olivier Nakache,Biograficzny,Francja,Top 2011 roku #1
4,Ojciec chrzestny,https://www.filmweb.pl/film/Ojciec+chrzestny-1...,"8,59",555 526,https://fwcdn.pl/fpo/10/89/1089/7196615_1.7.webp,The Godfather,1972,Opowieść o nowojorskiej rodzinie mafijnej. Sta...,2h 55m,Francis Ford Coppola,Mario Puzo,Dramat,USA,Top 1972 roku #1
5,Dwunastu gniewnych ludzi,https://www.filmweb.pl/film/Dwunastu+gniewnych...,"8,58",233 079,https://fwcdn.pl/fpo/07/01/30701/8094290_1.7.webp,12 Angry Men,1957,Dwunastu przysięgłych ma wydać wyrok w procesi...,1h 36m,Sidney Lumet,Reginald Rose,Dramat sądowy,USA,Top 1957 roku #1


>4. Przekształć czas trwania filmu w minuty.

In [41]:
import datetime
df['Duration'] = pd.to_datetime(df['Duration'], format='%H:%M').dt.time
df.head()

ValueError: time data "2h 22m" doesn't match format "%H:%M", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

>5. Jaka średnia długość filmów w TOP25?

>6. Jakiej produkcji są najczęściej filmy w TOP25?

>7. Filmy jakiej produkcji dostawały Oscar'a? Ile razy wg. TOP25?

>8. Inne ciekawe analizy na podstawie stworzonego zestawu danych (*opcjonalnie*).

>9. Podziel się wynikiem w grupie na [FB](https://bit.ly/2OSyHaG) podając hashtag `#modul8`.